# AI-Powered Recipe Retrieval System

## Problem Statement ❔
Given a prebuilt recipe dataset from Kaggle, how can we leverage AI to efficiently respond to user queries such as:


* Find me a recipe that takes 30 minutes⏳
* Show me a South Indian cuisine recipe 🍛
* List recipes with less than 5 ingredients 🍱
* What cuisines are available in the dataset? 📃

Manually filtering through large datasets is time-consuming. An AI-driven solution can automate query processing and retrieval, improving user experience.

## Solution 💡
This project uses Gemini’s function calling capabilities to convert natural language queries into structured database operations. The AI interprets user requests, applies fuzzy logic, and retrieves relevant recipes by interacting with predefined functions.

### Key Features:

* Dynamic Query Handling: Translates natural language into structured database filters.
* Efficient Retrieval: Quickly fetches recipes based on cuisine, cooking time, or ingredients.
* Interactive Responses: Provides formatted JSON responses for easy readability.

### Workflow 📝

1. **Predefined Functions**:
* describe_columns() → Lists dataset columns and descriptions.filter_recipes(condition)
* Filters recipes using boolean logic (e.g., "Cuisine == 'Italian'")
* filterUniqueValuesOfColumn(column) → Retrieves unique values (e.g., all cuisines)

2. **Tool Integration**:
* Functions are injected into Gemini as tools, enabling AI to call them dynamically.

3. **Few-Shot Prompting**:
* The AI is given custom instructions to interpret user queries and map them to relevant functions.

4. **Execution & Response**:
* The AI processes the query, calls the appropriate function, and returns structured results.

### Concepts Used ⚙️

* **Function Calling** (Gemini API)
* **Few-Shot Prompting** (Instruction-based query handling)
* **Structured Data Retrieval** (Pandas DataFrame operations)


### Model Used 🦴

* gemini-2.0-flash (Optimized for fast, structured responses)

This system enhances recipe discovery by making it intuitive, fast, and AI-driven, improving accessibility for users searching for culinary inspiration. 🍽️

# Configuration & Set Up

## Package Installation and Gemini API configuration

In [1]:
# Remove conflicting packages from the Kaggle base environment.
!pip uninstall -qqy kfp jupyterlab libpysal thinc spacy fastai ydata-profiling google-cloud-bigquery google-generativeai

In [2]:
pip install google-generativeai pandas faiss-cpu python-dotenv numpy chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of opentelemetry-proto to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of opentelemetry-proto to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of opentelemetry-sdk to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of opentelemetry-sdk to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.4/155.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 49.0 MB/s eta 0:

In [3]:
pip install --upgrade google-generativeai google-auth google-api-python-client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.3/212.3 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 84.7 MB/s eta 0:00:00
  Attempting uninstall: google-auth
    Found existing installation: google-auth 2.27.0
    Uninstalling google-auth-2.27.0:
      Successfully uninstalled google-auth-2.27.0
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 2.160.0
    Uninstalling google-api-python-client-2.160.0:
      Successfully uninstalled google-api-python-client-2.160.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pandas-gbq 0.26.1 requires google-cloud-bigquery<4.0.0dev,>=3.4.2, which is not installed.
bigframes 1.36.0 requires google-cloud-bigquery[bqstorage,pandas]>=3.18.0, which is not installed.
bigquery-magics 0.5.0 requires google-cloud-bigquery<4.0.0dev,>=3.13.0,

In [4]:
from google import genai
import google.generativeai as genai
from google.generativeai.types import GenerationConfig

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:623: UserWarning: <built-in function any> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(


In [5]:
#Configure Gemini

from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
GOOGLE_API_KEY = user_secrets.get_secret("GOOGLE_API_KEY")


# Configure with the latest API
genai.configure(api_key=GOOGLE_API_KEY)

In [6]:
import os
import re
import pandas as pd
import json
import faiss
import numpy as np
from typing import List, Dict, Optional

In [7]:
# Verify available models
print("Available models:")
for m in genai.list_models():
    if 'generateContent' in m.supported_generation_methods:
        print(f"- {m.name}")

Available models:
- models/gemini-1.0-pro-vision-latest
- models/gemini-pro-vision
- models/gemini-1.5-pro-latest
- models/gemini-1.5-pro-001
- models/gemini-1.5-pro-002
- models/gemini-1.5-pro
- models/gemini-1.5-flash-latest
- models/gemini-1.5-flash-001
- models/gemini-1.5-flash-001-tuning
- models/gemini-1.5-flash
- models/gemini-1.5-flash-002
- models/gemini-1.5-flash-8b
- models/gemini-1.5-flash-8b-001
- models/gemini-1.5-flash-8b-latest
- models/gemini-1.5-flash-8b-exp-0827
- models/gemini-1.5-flash-8b-exp-0924
- models/gemini-2.5-pro-exp-03-25
- models/gemini-2.5-pro-preview-03-25
- models/gemini-2.5-flash-preview-04-17
- models/gemini-2.0-flash-exp
- models/gemini-2.0-flash
- models/gemini-2.0-flash-001
- models/gemini-2.0-flash-exp-image-generation
- models/gemini-2.0-flash-lite-001
- models/gemini-2.0-flash-lite
- models/gemini-2.0-flash-lite-preview-02-05
- models/gemini-2.0-flash-lite-preview
- models/gemini-2.0-pro-exp
- models/gemini-2.0-pro-exp-02-05
- models/gemini-exp

## Dataset loading and Pre-processing

In [8]:
# Extract the recipe dataset into a dataframe
import pandas as pd
df = pd.read_csv('/kaggle/input/recipes-dataset/recipes.csv')

In [9]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/recipes-dataset/recipes.csv


In [10]:
df.head(20)

,title,ingredients,instructions,cuisine,TotalTimeInMins,url
0,Masala Karela Recipe,"salt,amchur (dry mango powder),karela (bitter ...","To begin making the Masala Karela Recipe,de-se...",Indian,45,https://www.archanaskitchen.com/masala-karela-...
1,Spicy Tomato Rice (Recipe),"tomato,salt,chickpea lentils,green chilli,rice...","To make tomato puliogere, first cut the tomato...",South Indian Recipes,15,https://www.archanaskitchen.com/spicy-tomato-r...
2,Ragi Semiya Upma Recipe - Ragi Millet Vermicel...,"salt,rice vermicelli noodles (thin),asafoetida...","To begin making the Ragi Vermicelli Recipe, fi...",South Indian Recipes,50,https://www.archanaskitchen.com/ragi-vermicell...
3,Gongura Chicken Curry Recipe - Andhra Style Go...,"tomato,salt,ginger,sorrel leaves (gongura),fen...",To begin making Gongura Chicken Curry Recipe f...,Andhra,45,https://www.archanaskitchen.com/gongura-chicke...
4,Andhra Style Alam Pachadi Recipe - Adrak Chutn...,"tomato,salt,ginger,red chillies,curry,asafoeti...","To make Andhra Style Alam Pachadi, first heat ...",Andhra,30,https://www.archanaskitchen.com/andhra-style-a...
5,Pudina Khara Pongal Recipe (Rice and Lentils C...,"cashew nuts,salt,yellow moong dal (split),ging...","To begin making Pudina Khara Pongal Recipe, wa...",South Indian Recipes,30,https://www.archanaskitchen.com/pudina-khara-p...
6,Udupi Style Ash Gourd Coconut Curry Recipe,"salt,tamarind water,coconut,mustard seeds,jagg...",To begin making the Udupi Style Ash Gourd Coco...,Udupi,40,https://www.archanaskitchen.com/udupi-style-as...
7,Mexican Style Black Bean Burrito Recipe,"tomato,spring onion greens,hung curd (greek yo...","To begin making the Black Bean Burrito recipe,...",Mexican,40,https://www.archanaskitchen.com/black-bean-bur...
8,Spicy Crunchy Masala Idli Recipe,"tomato,salt,coriander (dhania) leaves,idli,red...","To prepare Spicy Crunchy Masala Idli Recipe, H...",South Indian Recipes,30,https://www.archanaskitchen.com/spicy-crunchy-...
9,Cauliflower Leaves Chutney (Recipe in Hindi),"tomato,salt,turmeric powder,red chillies,spoon...","To make cauliflower leaf chutney, first of all...",South Indian Recipes,25,https://www.archanaskitchen.com/cauliflower-le...


In [11]:
df = df.rename(columns={
'title' : 'RecipeName',
'ingredients' : 'Ingredients',
'instructions' : 'Instructions',
'cuisine' : 'Cuisine',
'url' : 'RecipeURL'
})

In [12]:
list(df.columns)

['RecipeName',
 'Ingredients',
 'Instructions',
 'Cuisine',
 'TotalTimeInMins',
 'RecipeURL']

In [13]:
# Contain a one line description of each columns
df_desc = pd.DataFrame({
    'Column Name': list(df.columns),
    'Description': ['Name of the recipe', 'List of ingredients to make the recipe','Instructions to make the recipe','Cuisine type','Time taken in minutes to make the recipe', 'Link to recipe url']
})
print(df_desc)

       Column Name                               Description
0       RecipeName                        Name of the recipe
1      Ingredients    List of ingredients to make the recipe
2     Instructions           Instructions to make the recipe
3          Cuisine                              Cuisine type
4  TotalTimeInMins  Time taken in minutes to make the recipe
5        RecipeURL                        Link to recipe url


# Function & Tool Definition

In [14]:
import google.generativeai as genai
from google.generativeai.types import GenerationConfig
import pandas as pd
import json


# 1. Define actual backend functions
def describe_columns(df: pd.DataFrame) -> None:  # Get Column Description
    """Display column names and their data types
    Args: df: Pandas DataFrame containing recipe data
    """
    if df.empty:
        print("No data available - dataframe is empty")
        return

    print("\n Recipe Data Columns:")
    for col in df.columns:
        print(f"- {col} ({df[col].dtype})")

def filter_recipes(df: pd.DataFrame, condition: str, max_results: int =5) -> Optional[str]:
    """Filters recipes based on condition """
    try:
        if df.empty:
            return json.dumps({"error": "No recipe data available"}, indent=2)
            
        filtered = df.query(condition)
        
        if filtered.empty:
            return json.dumps({"message": "No recipes match your criteria"}, indent=2)
            
        return filtered.head(max_results).to_json(orient="records", indent=2)    
    
    except Exception as e:
        print(f" Filtering error: {e}")
        return None

def filterUniqueValuesOfColumn(df: pd.DataFrame, column:str) -> str:
    """ Get unique values from a column"""
    if column not in df.columns:
        return json.dumps({"error": f"Column '{column}' does not exist"}, indent=4)
    unique_values = df[column].drop_duplicates().tolist()  # Get unique values
    return json.dumps({"Unique Values": unique_values}, indent=4)

In [15]:
# 2. Define tools with prop schemas
db_tools = [
   {
        "function_declarations": [
            {
                "name": "describe_columns",
                "description": "Lists available columns in the DataFrame",
                "parameters": {"type": "object", "properties": {}}
            },
            {
                "name": "filter_recipes",
                "description": "Filters recipes using conditions like 'Cuisine == \"Italian\" AND TotalTimeInMins < 30'",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "condition": {
                            "type": "string",
                            "description": "Query using columns: RecipeName, Ingredients, Cuisine, TotalTimeInMins"
                        }
                    },
                    "required": ["condition"]
                }
            },
            {
                "name": "filterUniqueValuesOfColumn",
                "description": "Gets unique values from columns like Cuisine or Ingredients",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "column": {
                            "type": "string", 
                            "description": "Column name from: RecipeName, Ingredients, Cuisine, TotalTimeInMins, RecipeURL"
                        }
                    },
                    "required": ["column"]
                }
            }
        ]
    } 
]

In [16]:
# 3. Instruction to process to a user related query into set of functions and find relevant info.

instruction = """You are a helpful recipe assistant working with these columns:
- RecipeName: Name of the recipe
- Ingredients: List of ingredients
- Cuisine: Type of cuisine (e.g., Italian, Mexican)
- TotalTimeInMins: Total preparation time in minutes
- RecipeURL: Link to full recipe

Use these functions to help users:
1. describe_columns() → Show available columns
2. filter_recipes(condition) → Filter using conditions like "Cuisine == 'Italian'"
3. filterUniqueValuesOfColumn(column) → Get unique values from specific columns
"""

# Model Initialization and Chat Handling

In [17]:
# 4. Initialize Model

model = genai.GenerativeModel(
    'gemini-2.0-flash', 
    system_instruction=instruction, 
    tools=db_tools,
    generation_config=GenerationConfig(
        temperature=0.7, 
        top_p=0.9,
        top_k=40
    )
) 


In [18]:
# 5. Chat Handling with column name updates
def execute_query(query: str):
    response = chat.send_message(query)
    
    if response.candidates and response.candidates[0].content.parts[0].function_call:
        function_call = response.candidates[0].content.parts[0].function_call
        
        if function_call.name == "describe_columns":
            result = describe_columns(df)
        elif function_call.name == "filter_recipes":
            result = filter_recipes(df, function_call.args["condition"])
        elif function_call.name == "filterUniqueValuesOfColumn":
            result = filterUniqueValuesOfColumn(df, function_call.args["column"])
        else:
            result = json.dumps({"error": "Unknown function"})

        response = chat.send_message(
            genai.protos.Content(
                parts=[
                    genai.protos.Part(
                        function_response=genai.protos.FunctionResponse(
                            name=function_call.name,
                            response={"content": result}
                        )
                    )
                ]
            )
        )
    
    return response.text

# Chat Initialization and Sample User Interactions

In [19]:
# 6. Start chat   
chat = model.start_chat()

In [20]:
# 7a. Example Usage
print(execute_query("List down 5 different cuisines present in the recipe database"))

Here are 5 cuisines from the database: Indian, Mexican, Italian Recipes, Thai, Chinese.


In [21]:
# 7b. Example Usage
print(execute_query("Do you have a recipe that has south indian cuisine"))

Yes, I found 5 recipes with South Indian cuisine: Spicy Tomato Rice (Recipe), Ragi Semiya Upma Recipe - Ragi Millet Vermicelli Breakfast, Pudina Khara Pongal Recipe (Rice and Lentils Cooked With Mint Leaves & Mild Spices), Spicy Crunchy Masala Idli Recipe, and Cauliflower Leaves Chutney (Recipe in Hindi).


In [22]:
# 7c. Example Usage
print(execute_query("List any 5 Udipi cuisines"))

Here are 5 Udupi cuisine recipes: Udupi Style Ash Gourd Coconut Curry Recipe, Udupi Style Red Parboiled Rice Sevai Recipe (Red Rice Vermicelli), Padipe Saasmi Recipe - Udupi Style Amaranth Leaves Curry With Coconut & Tamarind, Udupi Red Parboiled Rice Shavige Recipe, and Kottambari Tambuli Recipe - Udupi Style Raw Coriander Kadhi.


In [23]:
# 7d. Example Usage
print(execute_query("List the preparation instructions for Udupi Style Ash Gourd Coconut Curry Recipe"))

Here are the instructions to prepare Udupi Style Ash Gourd Coconut Curry Recipe:

To begin making the Udupi Style Ash Gourd Coconut Curry Recipe cook the ash gourd in the pressure cooker with little water, turmeric powder and salt for just 1 whistle.
Release the pressure naturally.Soak the tamarind ball in hot water for about 15 minutes, and squeeze and keep the water.Then dry roast the spices and other ingredients for the ground paste in a pan except the coconut.
Then grind it along with coconut to a smooth paste.In same pressure cooker which has the ash gourd add the paste and little water and give it stir.
Keep the Udupi Ash Gourd Coconut Curry on the medium heat, Adjust the seasoning and texture if required.
Once done transfer it in a bowl.For the seasoning, heat oil in a small skillet and add mustard seeds and let it crackle.
After the mustard seeds have crackled add the curry leaves and red chillies.
Tun off the heat after the red chillies have darkened and add it on top of the U

In [24]:
# 7e. Example Usage
print(execute_query("Provide the link to full Recipe for Udupi Style Ash Gourd Coconut Curry Recipe"))

The link to the full recipe is https://www.archanaskitchen.com/udupi-style-ash-gourd-coconut-curry-recipe.
